In [56]:
from gensim import corpora, models, similarities
import gensim

In [57]:
import json 
import gzip 
#from StringIO import StringIO
import urllib.request
import requests, zipfile, io
import re

def parse(url): 
    if re.match("^http", url):
        r = requests.get(url)
        g = gzip.open(io.BytesIO(r.content))
    else:
        g = gzip.open(url, 'r') 
    for l in g:
        yield eval(l)

import json 
import gzip 
def parse(url): 
    g = gzip.open(url, 'r') 
    for l in g: 
        yield eval(l)

In [58]:
import gensim
import nltk
stoplist = nltk.corpus.stopwords.words('english')
lemma = nltk.stem.wordnet.WordNetLemmatizer()

def myReviews(review_files_urls, catFilter = None, asin = None, concatenate = False, max_count = None):
    preprocess_reviewText = []
    for url in review_files_urls:
        print("Processing url: " + url)
        review_item_iterator = iter(parse(url))
        it = 0
        conc_count = 0
        for review_item in review_item_iterator:
            if review_item['helpful'][1] == 0:
                review_item['percent_helpful'] = 0
            else:
                review_item['percent_helpful'] = review_item['helpful'][0] /  review_item['helpful'][1]
            review_item['review_helpful'] = (review_item['percent_helpful'] > .7) & (review_item['helpful'][1] > 5)
            if not review_item['review_helpful']:
                #print("Dropped because: percent_helpful: "+str(review_item['percent_helpful'])+" and helpful votes: "+str(review_item['helpful'][1]))  
                continue
            it += 1
            # max_count reached then stop generating values
            if (max_count is not None) and (it > max_count):
                return
            if it%100000==0:
                print("it="+str(it))

            reviewText = review_item["reviewText"]
            label = review_item["asin"] + "_" + review_item["reviewerID"] 
            preprocess_reviewText_i = gensim.utils.simple_preprocess(reviewText)
            preprocess_reviewText_i = [lemma.lemmatize(word) for word in preprocess_reviewText_i if word not in stoplist]


            #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
            if (catFilter == None or (catFilter in review_item["categories"])) and (asin == None or (asin == review_item["asin"])):
                if concatenate==False:
                    yield preprocess_reviewText_i
                else:
                    conc_count += 1
                    preprocess_reviewText.extend(preprocess_reviewText_i)
                    #print("Concatenated count: "+str(conc_count))
    if concatenate == True:
        print("Return the final text with length: "+str(len(preprocess_reviewText)))
        print(preprocess_reviewText)
        yield preprocess_reviewText

#base_url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/"
base_url = "/media/mister/ntfs/Rainer/y2buy/download/amazon_reviews_96_14/"

from bs4 import BeautifulSoup
import urllib.request


html = urllib.request.urlopen("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/")
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())
all_html_links = soup.find_all('a')
review_file_names = [links["href"] for links in all_html_links if re.match("^review.*[^_5|_10].json.gz$", links["href"])]

In [59]:
  
import os
dictionaryFileName = '../../download/word2vec/reviews_all.dict'
dictionary = corpora.Dictionary.load(dictionaryFileName)



In [60]:
def myCorpus(review_file_urls, catFilter = None, asin = None, concatenate = False, max_count = None):
    for review in myReviews(review_file_urls,catFilter,asin,concatenate,max_count):
    # assume there's one document per line, tokens separated by whitespace
        yield dictionary.doc2bow(review)

In [61]:
import os
tfidfFileName ='../../download/word2vec/reviews_all_corpus.tfidf'
tfidf = models.TfidfModel.load(tfidfFileName)

print(tfidf)

TfidfModel(num_docs=1093393, num_nnz=41632789)


In [62]:
import pickle

#tfidf_document_FileName ='../../download/word2vec/reviews_cell_phones_review.tfidf'
tfidf_document_FileName ='../../download/word2vec/reviews_cell_phones_only_helpful.corpus'
if os.path.isfile(tfidf_document_FileName)==False:
    document = next(myCorpus(review_file_urls = ['../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz'], catFilter="Cell Phones",concatenate=True))
    document_tfidf = tfidf[document]  
    pickle.dump(document_tfidf, open(tfidf_document_FileName, 'wb'))
else:
    document_tfidf = pickle.load( open( tfidf_document_FileName, "rb" ) )
    
    


Processing url: ../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [63]:
import gensim
model = gensim.models.Word2Vec.load('../../download/word2vec/myamazonmodel.word2vec')
#model = gensim.models.KeyedVectors.load_word2vec_format('../../download/word2vec/GoogleNews-vectors-negative300.bin', binary=True, limit=400000) 

In [64]:
import numpy as np
import pandas as pd
tbl =[[dictionary.get(tokenid),tfidf_value ,model.wv[dictionary.get(tokenid)] ] for (tokenid,tfidf_value) in document_tfidf if tfidf_value > 0.04 and dictionary.get(tokenid) in model.wv]

df = pd.DataFrame(data=tbl,
                  columns=["token","tfidf", "word_vector"]
                  )
df = df.sort_values('tfidf', ascending=False)
df.set_index("token")
print(df)

         token     tfidf                                        word_vector
21       phone  0.789646  [-1.04809, -1.02451, -0.955498, -0.361344, 0.3...
31     android  0.111846  [-3.92958, -0.211284, -0.239311, 4.65345, 0.07...
23      screen  0.104127  [-1.17817, -2.13697, 0.909865, 2.24479, 0.3716...
37        apps  0.103336  [-0.17236, 0.09563, 0.858206, 1.95059, -0.3707...
18        call  0.102417  [-0.33118, -3.82637, 3.85393, 0.325647, 3.1811...
24      mobile  0.098812  [-1.08778, 1.74741, 1.47753, 2.00216, 0.639684...
41         sim  0.095065  [0.594038, -1.7372, 1.23972, -1.81601, 0.34559...
25       nokia  0.091546  [-1.18261, -0.477743, -0.686531, 2.5095, -0.90...
29      iphone  0.077432  [-2.20164, -0.346402, -1.60772, 0.869051, 0.11...
22     battery  0.071330  [-4.26823, 0.74218, 1.45559, 2.85068, -2.32551...
42    unlocked  0.063498  [-3.4626, -2.06992, 1.08157, -0.139683, 0.8514...
10        text  0.063462  [-2.60979, -1.7846, 1.99294, 0.919722, 0.50104...
27     samsu

In [65]:
import nltk
from nltk.cluster.kmeans import KMeansClusterer
nc = 10
data = df.loc[:,"word_vector"]
#print(data)
kclusterer = KMeansClusterer(nc, distance=nltk.cluster.util.cosine_distance, avoid_empty_clusters=True) #repeats=30,
assigned_clusters = kclusterer.cluster(data, assign_clusters=True)
#print(assigned_clusters)
means = np.asarray( kclusterer.means())


In [66]:
df["cluster"] = pd.Series(assigned_clusters, index=df.index)

print(df.shape)
print(df)

(47, 4)
         token     tfidf                                        word_vector  \
21       phone  0.789646  [-1.04809, -1.02451, -0.955498, -0.361344, 0.3...   
31     android  0.111846  [-3.92958, -0.211284, -0.239311, 4.65345, 0.07...   
23      screen  0.104127  [-1.17817, -2.13697, 0.909865, 2.24479, 0.3716...   
37        apps  0.103336  [-0.17236, 0.09563, 0.858206, 1.95059, -0.3707...   
18        call  0.102417  [-0.33118, -3.82637, 3.85393, 0.325647, 3.1811...   
24      mobile  0.098812  [-1.08778, 1.74741, 1.47753, 2.00216, 0.639684...   
41         sim  0.095065  [0.594038, -1.7372, 1.23972, -1.81601, 0.34559...   
25       nokia  0.091546  [-1.18261, -0.477743, -0.686531, 2.5095, -0.90...   
29      iphone  0.077432  [-2.20164, -0.346402, -1.60772, 0.869051, 0.11...   
22     battery  0.071330  [-4.26823, 0.74218, 1.45559, 2.85068, -2.32551...   
42    unlocked  0.063498  [-3.4626, -2.06992, 1.08157, -0.139683, 0.8514...   
10        text  0.063462  [-2.60979, -1.7846

In [67]:
df["centroid"] = pd.Series( means[assigned_clusters].tolist(), index=df.index)

print(df.shape)
print(df)


(47, 5)
         token     tfidf                                        word_vector  \
21       phone  0.789646  [-1.04809, -1.02451, -0.955498, -0.361344, 0.3...   
31     android  0.111846  [-3.92958, -0.211284, -0.239311, 4.65345, 0.07...   
23      screen  0.104127  [-1.17817, -2.13697, 0.909865, 2.24479, 0.3716...   
37        apps  0.103336  [-0.17236, 0.09563, 0.858206, 1.95059, -0.3707...   
18        call  0.102417  [-0.33118, -3.82637, 3.85393, 0.325647, 3.1811...   
24      mobile  0.098812  [-1.08778, 1.74741, 1.47753, 2.00216, 0.639684...   
41         sim  0.095065  [0.594038, -1.7372, 1.23972, -1.81601, 0.34559...   
25       nokia  0.091546  [-1.18261, -0.477743, -0.686531, 2.5095, -0.90...   
29      iphone  0.077432  [-2.20164, -0.346402, -1.60772, 0.869051, 0.11...   
22     battery  0.071330  [-4.26823, 0.74218, 1.45559, 2.85068, -2.32551...   
42    unlocked  0.063498  [-3.4626, -2.06992, 1.08157, -0.139683, 0.8514...   
10        text  0.063462  [-2.60979, -1.7846

In [68]:
df["centroid_distance"] = df.apply(lambda row:nltk.cluster.util.cosine_distance(row['word_vector'], row['centroid']), axis=1)

print(df.shape)
print(df)

(47, 6)
         token     tfidf                                        word_vector  \
21       phone  0.789646  [-1.04809, -1.02451, -0.955498, -0.361344, 0.3...   
31     android  0.111846  [-3.92958, -0.211284, -0.239311, 4.65345, 0.07...   
23      screen  0.104127  [-1.17817, -2.13697, 0.909865, 2.24479, 0.3716...   
37        apps  0.103336  [-0.17236, 0.09563, 0.858206, 1.95059, -0.3707...   
18        call  0.102417  [-0.33118, -3.82637, 3.85393, 0.325647, 3.1811...   
24      mobile  0.098812  [-1.08778, 1.74741, 1.47753, 2.00216, 0.639684...   
41         sim  0.095065  [0.594038, -1.7372, 1.23972, -1.81601, 0.34559...   
25       nokia  0.091546  [-1.18261, -0.477743, -0.686531, 2.5095, -0.90...   
29      iphone  0.077432  [-2.20164, -0.346402, -1.60772, 0.869051, 0.11...   
22     battery  0.071330  [-4.26823, 0.74218, 1.45559, 2.85068, -2.32551...   
42    unlocked  0.063498  [-3.4626, -2.06992, 1.08157, -0.139683, 0.8514...   
10        text  0.063462  [-2.60979, -1.7846

In [69]:
writer = pd.ExcelWriter('clustered_cellphone_features_only_helpful.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

print(tfidf)